In [1]:
import tensorflow as tf
import numpy as np
import keras
import glob
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os

# Percorsi locali su Kaggle (modifica il nome della directory secondo la tua struttura!)
image_dir = '/kaggle/input/images'       # <-- Modifica se il nome della cartella è diverso
mask_dir = '/kaggle/input/labels-index'  # <-- Modifica se il nome della cartella è diverso

# Trova tutte le immagini e maschere
image_paths = sorted(glob.glob(os.path.join(image_dir, '*.png')))
mask_paths = sorted(glob.glob(os.path.join(mask_dir, '*.npy')))

# Split: train / val / test
train_img, temp_img, train_mask, temp_mask = train_test_split(
    image_paths, mask_paths, test_size=0.3, random_state=42)

val_img, test_img, val_mask, test_mask = train_test_split(
    temp_img, temp_mask, test_size=0.5, random_state=42)

# Funzione per caricare immagine e maschera
def load_image_mask(image_path, mask_path):
    # Carica immagine PNG
    img = tf.io.read_file(image_path)
    img = tf.image.decode_png(img, channels=3)
    img = tf.cast(img, tf.float32) / 255.0
    img = tf.image.resize(img, [720, 960])  # Resize opzionale

    # Carica maschera .npy con numpy_function
    def load_npy(np_path):
        return np.load(np_path.decode('utf-8')).astype(np.uint8)

    mask = tf.numpy_function(load_npy, [mask_path], tf.uint8)
    mask.set_shape([None, None])  # Serve per tf.data
    mask = tf.image.resize(mask[..., tf.newaxis], [720, 960], method='nearest')
    mask = tf.squeeze(mask, axis=-1)

    return img, mask


2025-07-09 14:21:08.296126: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752070868.668486      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752070868.770324      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
class InstanceNormalization(keras.layers.Layer):
    def __init__(self, epsilon=1e-5, **kwargs):
        super(InstanceNormalization, self).__init__(**kwargs)
        self.epsilon = epsilon

    def build(self, input_shape):
        dim = input_shape[-1]
        self.gamma = self.add_weight(
            shape=(dim,),
            initializer="ones",
            trainable=True,
            name="gamma"
        )
        self.beta = self.add_weight(
            shape=(dim,),
            initializer="zeros",
            trainable=True,
            name="beta"
        )

    def call(self, inputs):
        mean, variance = tf.nn.moments(inputs, axes=[1, 2], keepdims=True)
        normalized = (inputs - mean) / tf.sqrt(variance + self.epsilon)
        return self.gamma * normalized + self.beta

In [3]:
num_classes = 32
inputs = Input(shape=(720, 960, 3))

# Encoder
c1 = Conv2D(16, 3, padding='same')(inputs)
c1 = InstanceNormalization()(c1)
c1 = Activation('relu')(c1)
c1 = Conv2D(16, 3, padding='same')(c1)
c1 = InstanceNormalization()(c1)
c1 = Activation('relu')(c1)
c1 = SpatialDropout2D(0.2)(c1)
p1 = MaxPooling2D(2)(c1)  # 360x480

c2 = Conv2D(32, 3, padding='same')(p1)
c2 = InstanceNormalization()(c2)
c2 = Activation('relu')(c2)
c2 = Conv2D(32, 3, padding='same')(c2)
c2 = InstanceNormalization()(c2)
c2 = Activation('relu')(c2)
c2 = SpatialDropout2D(0.2)(c2)
p2 = MaxPooling2D(2)(c2)  # 180x240

c3 = Conv2D(64, 3, padding='same')(p2)
c3 = InstanceNormalization()(c3)
c3 = Activation('relu')(c3)
c3 = Conv2D(64, 3, padding='same')(c3)
c3 = InstanceNormalization()(c3)
c3 = Activation('relu')(c3)
c3 = SpatialDropout2D(0.2)(c3)
p3 = MaxPooling2D(2)(c3)  # 90x120

c4 = Conv2D(128, 3, padding='same')(p3)
c4 = InstanceNormalization()(c4)
c4 = Activation('relu')(c4)
c4 = Conv2D(128, 3, padding='same')(c4)
c4 = InstanceNormalization()(c4)
c4 = Activation('relu')(c4)
c4 = SpatialDropout2D(0.2)(c4)
p4 = MaxPooling2D(2)(c4)  # 45x60

# Bottleneck
b = Conv2D(256, 3, padding='same')(p4)
b = InstanceNormalization()(b)
b = Activation('relu')(b)
b = Conv2D(256, 3, padding='same')(b)
b = InstanceNormalization()(b)
b = Activation('relu')(b)
b = SpatialDropout2D(0.2)(b)

# Decoder con Attention Gates
u1 = Conv2DTranspose(128, 3, strides=2, padding='same')(b)  # 90x120
attn1 = attention_gate(c4, u1, inter_channels=64)
u1 = Concatenate()([u1, attn1])
c5 = Conv2D(128, 3, padding='same')(u1)
c5 = InstanceNormalization()(c5)
c5 = Activation('relu')(c5)
c5 = Conv2D(128, 3, padding='same')(c5)
c5 = InstanceNormalization()(c5)
c5 = Activation('relu')(c5)
c5 = SpatialDropout2D(0.2)(c5)

u2 = Conv2DTranspose(64, 3, strides=2, padding='same')(c5)  # 180x240
attn2 = attention_gate(c3, u2, inter_channels=32)
u2 = Concatenate()([u2, attn2])
c6 = Conv2D(64, 3, padding='same')(u2)
c6 = InstanceNormalization()(c6)
c6 = Activation('relu')(c6)
c6 = Conv2D(64, 3, padding='same')(c6)
c6 = InstanceNormalization()(c6)
c6 = Activation('relu')(c6)
c6 = SpatialDropout2D(0.2)(c6)

u3 = Conv2DTranspose(32, 3, strides=2, padding='same')(c6)  # 360x480
attn3 = attention_gate(c2, u3, inter_channels=16)
u3 = Concatenate()([u3, attn3])
c7 = Conv2D(32, 3, padding='same')(u3)
c7 = InstanceNormalization()(c7)
c7 = Activation('relu')(c7)
c7 = Conv2D(32, 3, padding='same')(c7)
c7 = InstanceNormalization()(c7)
c7 = Activation('relu')(c7)
c7 = SpatialDropout2D(0.2)(c7)

u4 = Conv2DTranspose(16, 3, strides=2, padding='same')(c7)  # 720x960
attn4 = attention_gate(c1, u4, inter_channels=8)
u4 = Concatenate()([u4, attn4])
c8 = Conv2D(16, 3, padding='same')(u4)
c8 = InstanceNormalization()(c8)
c8 = Activation('relu')(c8)
c8 = Conv2D(16, 3, padding='same')(c8)
c8 = InstanceNormalization()(c8)
c8 = Activation('relu')(c8)
c8 = SpatialDropout2D(0.2)(c8)

# Output layer
outputs = Conv2D(num_classes, 1, activation='softmax', padding='same')(c8)

model = keras.Model(inputs, outputs)
model.summary()

I0000 00:00:1752070888.207917      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1752070888.208749      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 720, 960,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 720, 960,  │        448 │ input_layer[0][0] │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ instance_normaliza… │ (None, 720, 960,  │         32 │ conv2d[0][0]      │
│ (InstanceNormaliza… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 720, 960,  │          0 │ instance_normali… │
│ (Activation)        │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 720, 960,  │      2,320 │ activation[0][0]  │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ instance_normaliza… │ (None, 720, 960,  │         32 │ conv2d_1[0][0]    │
│ (InstanceNormaliza… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 720, 960,  │          0 │ instance_normali… │
│ (Activation)        │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ spatial_dropout2d   │ (None, 720, 960,  │          0 │ activation_1[0][… │
│ (SpatialDropout2D)  │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 360, 480,  │          0 │ spatial_dropout2… │
│ (MaxPooling2D)      │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 360, 480,  │      4,640 │ max_pooling2d[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ instance_normaliza… │ (None, 360, 480,  │         64 │ conv2d_2[0][0]    │
│ (InstanceNormaliza… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 360, 480,  │          0 │ instance_normali… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 360, 480,  │      9,248 │ activation_2[0][… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ instance_normaliza… │ (None, 360, 480,  │         64 │ conv2d_3[0][0]    │
│ (InstanceNormaliza… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 360, 480,  │          0 │ instance_normali… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ spatial_dropout2d_1 │ (None, 360, 480,  │          0 │ activation_3[0][… │
│ (SpatialDropout2D)  │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 180, 240,  │          0 │ spatial_dropout2

 Total params: 1,966,096 (7.50 MB)

 Trainable params: 1,966,096 (7.50 MB)

 Non-trainable params: 0 (0.00 B)

In [4]:
def create_dataset(img_paths, mask_paths, batch_size=8, shuffle=True):
    dataset = tf.data.Dataset.from_tensor_slices((img_paths, mask_paths))
    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(img_paths))
    dataset = dataset.map(load_image_mask, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset

batch_size = 8
train_ds = create_dataset(train_img, train_mask, batch_size)
val_ds = create_dataset(val_img, val_mask, batch_size, shuffle=False)
test_ds = create_dataset(test_img, test_mask, batch_size, shuffle=False)

In [5]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',  # perché la maschera è un indice
    metrics=['accuracy']
)

In [6]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=20
)

Epoch 1/20


I0000 00:00:1752070910.184580      99 service.cc:148] XLA service 0x7f3124039c80 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1752070910.186159      99 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1752070910.186184      99 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
W0000 00:00:1752070911.515244      99 assert_op.cc:38] Ignoring Assert operator compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert
I0000 00:00:1752070912.387324      99 cuda_dnn.cc:529] Loaded cuDNN version 90300
E0000 00:00:1752070915.381523      99 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1752070915.528276      99 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may 

61/62 ━━━━━━━━━━━━━━━━━━━━ 0s 963ms/step - accuracy: 0.3224 - loss: 2.8340

W0000 00:00:1752071042.947160      99 assert_op.cc:38] Ignoring Assert operator compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert
E0000 00:00:1752071045.149668      99 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1752071045.299331      99 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1752071045.440364      99 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1752071045.928783      99 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:175207

62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.3248 - loss: 2.8287   

W0000 00:00:1752071090.913015      97 assert_op.cc:38] Ignoring Assert operator compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert
W0000 00:00:1752071096.253081     100 assert_op.cc:38] Ignoring Assert operator compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert
E0000 00:00:1752071096.564433     100 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1752071096.699118     100 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1752071097.225033     100 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1752071097.363

62/62 ━━━━━━━━━━━━━━━━━━━━ 212s 2s/step - accuracy: 0.3271 - loss: 2.8237 - val_accuracy: 0.6387 - val_loss: 1.9702
Epoch 2/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 70s 1s/step - accuracy: 0.6303 - loss: 1.9227 - val_accuracy: 0.6740 - val_loss: 1.5043
Epoch 3/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 69s 1s/step - accuracy: 0.6674 - loss: 1.5268 - val_accuracy: 0.6980 - val_loss: 1.2660
Epoch 4/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 69s 1s/step - accuracy: 0.6918 - loss: 1.3261 - val_accuracy: 0.7325 - val_loss: 1.0883
Epoch 5/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 69s 1s/step - accuracy: 0.7147 - loss: 1.1876 - val_accuracy: 0.7398 - val_loss: 1.0395
Epoch 6/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 69s 1s/step - accuracy: 0.7253 - loss: 1.1291 - val_accuracy: 0.7596 - val_loss: 0.9517
Epoch 7/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 69s 1s/step - accuracy: 0.7377 - loss: 1.0635 - val_accuracy: 0.7754 - val_loss: 0.8868
Epoch 8/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 69s 1s/step - accuracy: 0.7537 - loss: 0.9892 - val_accuracy: 0.7766 - val_loss: 0.8664
Ep